# 1. Import packages

In [184]:
from bs4 import BeautifulSoup # html을 soup에 넣고 특정 class or data를 가져오는 library
import re
import urllib
import requests
import shutil
import collections
import codecs
import json
from selenium import webdriver
import pandas as pd
import numpy as np
import os

In [241]:
def hangul(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub('', text) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    return str(result)


def find_profile_img():
    test = driver.find_elements_by_tag_name("img")
    for i in test:
        if blog_id in i.get_attribute("src"): 
            blog_profile_img = i.get_attribute("src")
            break
    return blog_profile_img

def find_categories():
    test = driver.find_elements_by_css_selector("body > div.lyr_category_lst1 > ul > li:nth-child(1) > ul > li > a")
    Categories = []
    for i in test:
        Categories.append(i.text.split("\n")[:2])
    Count_Categories = len(Categories)
    return Categories, Count_Categories

def find_post_id(post):
    r = re.compile('logNo=.*')
    string = r.findall(post.get_attribute('ng-href'))
    return re.sub("logNo=",'',"".join(string))

# 2. Read_data

In [186]:
#driver = webdriver.PhantomJS('.//Exe_file//phantomjs.exe')
driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
sample_data = pd.read_csv('Test_data.txt',encoding='euc-kr')

In [187]:
sample_data.head()

,ID,Post_ID,Category,Crediblity,Source
0,yjoshjh,221296055589,맛집,1,Naver
1,emptyh,221284843491,맛집,1,Naver
2,youngchayoungcha,221183788830,어학·외국어,1,Naver
3,angelpyj2,221294255542,어학·외국어,1,Naver
4,seoulvitamin,220218119352,건강·의학,1,Naver


# 3.Make csv_file

In [156]:
csv_name = 'Naver_Crawling_text.csv'
if not os.path.exists(csv_name):
    with open('Naver_Crawling_text.csv', 'w') as f:
        f.write('')

# 4. Data_Crwaling

## User_Table Crwaling

In [321]:
User_id = sample_data['ID'][0]
Post_id = sample_data['Post_ID'][0]
Category = sample_data['Category'][0]
Source = sample_data['Source'][0]
credibility = sample_data['Crediblity'][0]
    
url = "http://blog.naver.com/PostView.nhn?blogId="+User_id+ "&logNo=" + Post_id +"&redirect=Dlog&widgetTypeCall=true"
mobile_url = "http://m.blog.naver.com/PostView.nhn?blogId="+blog_id

In [297]:
driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
driver.get(mobile_url)
driver.find_element_by_class_name("btn_area").click()

In [356]:
# get mobile_url

driver = webdriver.Chrome('.//exe_file//chromedriver.exe')
driver.get(mobile_url)

# 존재하지 않는 게시물 클릭
driver.find_element_by_class_name("btn_area").click()
driver.implicitly_wait(3)

# Blog_name, Blog_nickname, Blog_mobile_profile_img, Blog_info_text
Blog_name = driver.find_element_by_css_selector('#rego_cover > div.cover_cont > div.tit_area > h2 > a > span').text
Blog_nickname =driver.find_element_by_class_name("user_name").text
Blog_mobile_profile_img = find_profile_img()
Blog_info_text = driver.find_element_by_class_name("text").text

# Count_neighbors
neighbors_string = re.sub(",","",driver.find_element_by_class_name("count_buddy").text)
Count_neighbors = int(re.findall('\d+', neighbors_string)[0])

# Count_visitors
visitor_stirng = driver.find_elements_by_class_name('count')[0].text
Count_visitors = re.sub(",","",visitor_stirng.split("전체")[1]).strip()

# download_img
Mobile_cover_img = driver.find_element_by_class_name("cover_img").get_attribute('bg-lazy-img')

# category 버튼 클릭
driver.find_element_by_css_selector("#rego_cover > div.cover_cont > div.btn_area > div > div:nth-child(2) > a > span.txt").click()
driver.implicitly_wait(3)

# Total_Post, Categories, Count_Categories, 

Categories, Count_categories =  find_categories()
Total_post = driver.find_element_by_class_name("num").text

In [ ]:
+-

In [362]:
# post

for i in driver.find_elements_by_class_name("thumb_link"):
    print(find_post_id(i))

221297882299
221297411013
221296890227
221296316274
221296055589
221295389068
221295041071
221294592109
221294518505
221293145488
221293080533
221291968255
221291472684
221291033906
221289450904
221289135362
221288350583
221287832536
221287378059
221286987853


In [361]:
driver.find_element_by_class_name("tit_h3").text

'[역삼동 맛집] 평양옥 : 연중무휴 평양냉면 전문 식당 - 평양냉면의 신흥 강자 - 깔끔한 곰탕도 맛있는'

## Post_Table Crwaling

In [176]:
print(main_url)

http://blog.naver.com/PostView.nhn?blogId=yjoshjh


In [177]:
print(mobile_url)

http://m.blog.naver.com/PostView.nhn?blogId=yjoshjh


## Tag_Table Crwaling

In [ ]:
http://blog.naver.com/PostList.nhn?blogId=yjoshjh

## Img_Tabe Crwaling

In [ ]:
for i in range(0,len(sample_data)):
    
    blog_id = sample_data['Id'][i]
    post_No = str(sample_data['post_No'][i]) 
    category = sample_data['Category'][i]
    source = sample_data['source'][i]
    credibility = sample_data['Crediblity'][i]
    
    url = "http://blog.naver.com/PostView.nhn?blogId="+blog_id+ "&logNo=" + post_No +"&redirect=Dlog&widgetTypeCall=true"
    
     # if check credible or Non_credible    
    if sample_data['Crediblity'][i] == 1:
        check = "Credible"
    else:
        check = "Non_Credible"
    move_folder = "C:\\Users\\AjouHCI\\Desktop\\Pic_sample\\" + check
    
    r = requests.get(url)
    bs = BeautifulSoup(r.text.encode("utf-8"), "html.parser")

    #title
    # Append_value
    title = bs.find("h3", {"class": "se_textarea"})
    if (title == None):
        title = bs.find("span", {"class": "pcol1 itemSubjectBoldfont"})
    if (title != None):
        title = title.text.strip()
    else:
        title = "TITLE ERROR"
    title = title.text

    #date
    # Append_value
    date = bs.find("span", {"class": "se_publishDate pcol2 fil5"})
    if date == None:
        date = bs.find("p",{"class":"date fil5 pcol2 _postAddDate"})
        
    date_text = re.sub("\n","",date.text)
    date = re.sub("\t","",date_text)

    #Teg, Content_structure, Text

    # structure
    structure = bs.find("div", {"id": "postViewArea"})
    if structure == None:
        structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})

    # structure tag
    structure_tag = structure.find_all(['p','img'])
    
    # content
    content_list = []
    for i in structure_tag:        
        if str(i)[:2] == "<p":
            content_list.append("<p>")
            check = i.find_all(['span','br'])
            for j in check:
                if str(j)[:2] == '<s':
                    if hangul(j.text.strip()) != '':
                        content_list.append(j)
                else:
                    content_list.append(j)
            content_list.append("</p>")

        if str(i)[:4] == "<img":
            content_list.append(i)
            
    # content_text
    text_list = []
    for i in content_list:
        try:
            text_list.append(i.text)
        except:
            pass
    
    # Append_value
    content_structre = "|".join([str(i) for i in content_list])
    text = " ".join(list(map(lambda x : x.strip(),list(filter(lambda x: len(x)>1 ,text_list)))))
    tag_structure = "|".join(list(map(lambda x: str(x)[:5].strip() +'>' if str(x)[:5][-1] != '>' else str(x)[:5] ,content_list)))

    #Category_list & open webdriver
    
    driver = webdriver.PhantomJS('.//Exe_file//phantomjs.exe')
    driver.get(url)
    driver.implicitly_wait(3)
    category = driver.find_element_by_class_name("cm-body")
    bs = BeautifulSoup(category.text.encode("utf-8"), "html.parser")
    
    try:
        total_post_count = int(re.sub('\D',"",bs.text.split("\n")[0].split(" ")[1]))
    except ValueError:
        category = driver.find_element_by_id("category-list")
        bs = BeautifulSoup(category.text.encode("utf-8"), "html.parser")
        total_post_count = int(re.sub('\D',"",bs.text.split("\n")[0].split(" ")[1]))
    
    category = bs.text.split("\n")[1:]
    category_count = len(category)
    driver.close()
    
    #Img downloads
    imgs = structure.find_all('img')
    
    ## make img category folder + user_id folder
    
    
    ############################################
    for num,img in enumerate(imgs):
        
        if "스티커 이미지" in str(i):
            savename = "sticker_" + blog_id + "_" + str(num) + ".jpg"
        else:
            img_url = re.sub("\u200b","",str(img['src']))
            savename =  blog_id + "_" + str(num) + ".jpg"
            map_exist = 0
        
        try:
            urllib.request.urlretrieve(img_url,savename)
            shutil.move(savename,move_folder)
        except UnicodeEncodeError:
            if 'map' not in str(img_url):
                driver = webdriver.PhantomJS('.//Exe_file//phantomjs.exe')
                driver.get(url)
                driver.implicitly_wait(3)
                img = driver.find_element_by_tag_name('img')
                src = re.sub("\u200b","",str(img.get_attribute('src')))
                urllib.request.urlretrieve(src, savename)
                shutil.move(savename,move_folder)
                driver.close()
            else:
                map_exist = 1

    
    # make a save list
    save_content = blog_id + "||" + post_No + "||" + category + "||" + category_count + title + "||" + date + "||" + text + "||" \
    + content_structre + "||" + tag_structure + "||" +  total_post_count + "||" + category_count + "||" + map_exist

    # save to json
    with open('Naver_Crawling_text.csv', 'a') as f:
        f.write(save_content + "\n")

# 4. load_json_file

# 방문자 숫자

In [ ]:
mobile_url = 

In [29]:
blog_id = "ghatyvlddl1"
post_No = str(221291478698) 

url = "http://blog.naver.com/PostView.nhn?blogId="+blog_id+ "&logNo=" + post_No +"&redirect=Dlog&widgetTypeCall=true"

In [39]:
url

'http://blog.naver.com/PostView.nhn?blogId=ghatyvlddl1&logNo=221291478698&redirect=Dlog&widgetTypeCall=true'

In [33]:
r = requests.get(url)
bs = BeautifulSoup(r.text.encode("utf-8"), "html.parser")
structure = bs.find("div", {"id": "postViewArea"})
if structure == None:
    structure = bs.find("div",{"class","se_component_wrap sect_dsc __se_component_area"})
imgs = structure.find_all('img')

In [38]:
for i in imgs:
    if "스티커 이미지" in str(i):
        print(i)

<img alt="스티커 이미지" class="__se_img_el" height="147" src="https://storep-phinf.pstatic.net​/brown_and_cony/original_65.png?type=p50_50" width="137"/>
<img alt="스티커 이미지" class="__se_img_el" height="132" src="https://storep-phinf.pstatic.net​/brown_and_cony/original_43.png?type=p50_50" width="183"/>
<img alt="스티커 이미지" class="__se_img_el" height="150" src="https://storep-phinf.pstatic.net​/line_characters_in_love/original_29.png?type=p50_50" width="155"/>


In [45]:
for i in range(0,len(sample_data)):
    blog_id = sample_data['ID'][i]
    m_url = "https://m.blog.naver.com/PostList.nhn?blogId=" + blog_id
    print(m_url)

https://m.blog.naver.com/PostList.nhn?blogId=yjoshjh
https://m.blog.naver.com/PostList.nhn?blogId=emptyh
https://m.blog.naver.com/PostList.nhn?blogId=youngchayoungcha
https://m.blog.naver.com/PostList.nhn?blogId=angelpyj2
https://m.blog.naver.com/PostList.nhn?blogId=seoulvitamin
https://m.blog.naver.com/PostList.nhn?blogId=ra7575
https://m.blog.naver.com/PostList.nhn?blogId=namu0808
https://m.blog.naver.com/PostList.nhn?blogId=darkbior
https://m.blog.naver.com/PostList.nhn?blogId=thkim0511
https://m.blog.naver.com/PostList.nhn?blogId=rockview
https://m.blog.naver.com/PostList.nhn?blogId=b852d
https://m.blog.naver.com/PostList.nhn?blogId=sdhee99
https://m.blog.naver.com/PostList.nhn?blogId=dkla062500
https://m.blog.naver.com/PostList.nhn?blogId=ljc715555
https://m.blog.naver.com/PostList.nhn?blogId=shawnlover
https://m.blog.naver.com/PostList.nhn?blogId=ghatyvlddl1
https://m.blog.naver.com/PostList.nhn?blogId=blacktea1015
https://m.blog.naver.com/PostList.nhn?blogId=sbni1045
https://m.b

In [ ]:
m_url = "https://m.blog.naver.com/PostList.nhn?blogId=" + blog_id